This is based off Theo's multipletargets notebook. Have many targets want to see top diseases for these targets.

In [21]:

import requests
import random
import pandas as pd
import uuid
import json
from tabulate import tabulate

In [22]:
# This is a useful trick to explore API responses from https://github.com/caldwell/renderjson
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 300px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)


In [23]:
API = 'http://localhost:8008/api/latest'

# Find what's known for a list of genes

> eg. Maria L in the fly group routinely screens lists of human genes for the presence of evidence in neurological diseases. She wants to understand a bit about the biology of these genes. She looks at open targets to find what is known about the target and to which disease it has already been connected.

Compiling evidence relative to a list of genes can be a time-consuming and tedious task. Now the process can be automated, thanks to the targetvalidation.org API.

This tutorial illustrates how to compile all the evidence contained in targetvalidation.org for a list of genes into a compact representation (eg. a spreadsheet) using python. 
You might have a list of genes being generated from an experiment or a genetic screen, or perhaps you are following a particular set of targets and would like to maintain an updated look at what evidence has been compiled around them.
The process can be useful to reduce bias and increase success rates when prioritizing a list of genes for experimental follow-up. 

For this tutorial, we will use a list of random genes taken from the HGNC catalog. First, let's download the full catalog and create a list of symbols only.

In [24]:
%%capture
%%bash
[ -f ~/Documents/CTTV/hgnc_complete_set.txt.gz ] || wget --directory-prefix=../data/ http://ftp.ebi.ac.uk/pub/databases/genenames/hgnc_complete_set.txt.gz 
[ -f ~/Documents/CTTV/hgnc_complete_set.txt ] || gunzip ../data/hgnc_complete_set.txt.gz 
tail -n +2 ~/Documents/CTTV/hgnc_complete_set.txt | cut -f2 | grep -v withdrawn > ~/Documents/CTTV/hgnc_symbol_set.txt

We then print ten random gene names to check that our shuffling is working.

In [25]:
import random
with open('/Users/rgaspary/Documents/CTTV/hgnc_symbol_set.txt') as f:
    genes = [line.rstrip() for line in f]
random.shuffle(genes)
print(genes[:10])

['DPPA5P4', 'EMC10', 'BRD7P6', 'HK3', 'CRX', 'HOMER2P1', 'CADM3-AS1', 'C1orf123', 'GAGE7', 'GFRA3']


To use them in querying the Target Validation API we have to convert them to Ensembl gene ids:

In [27]:
def get_ensid(genesymbol):
    '''
    uses the targetvalidation.org API to get ENS IDs
    '''
    r = requests.get(API + '/public/search', 
                     params={'q':genesymbol,'size':1,'filter':'gene'})
    result = r.json()
    #print(result)
    try:
        if result["data"][0]["data"]["approved_symbol"] == genesymbol:
            geneId = result["data"][0]["id"]
            return geneId
    except IndexError:
            return None

assert get_ensid('SOD1') == 'ENSG00000142168'

random_genes = [get_ensid(x) for x in genes[:10] if get_ensid(x) is not None]
print(random_genes)

{u'from': 0, u'facets': None, u'took': 39, u'therapeutic_areas': [], u'total': 12, u'data': [{u'highlight': {u'ortholog.yeast.name': [u'superoxide dismutase <em>SOD1</em>'], u'ortholog.chimpanzee.symbol': [u'<em>SOD1</em>', u'<em>SOD1</em>', u'<em>SOD1</em>'], u'approved_symbol': [u'<em>SOD1</em>'], u'ortholog.mouse.symbol': [u'<em>Sod1</em>'], u'ortholog.zebrafish.symbol': [u'<em>sod1</em>'], u'ortholog.dog.symbol': [u'<em>SOD1</em>'], u'ortholog.frog.symbol': [u'<em>sod1</em>'], u'ortholog.yeast.symbol': [u'<em>SOD1</em>'], u'ortholog.rat.symbol': [u'<em>Sod1</em>'], u'ortholog.macaque.symbol': [u'<em>SOD1</em>'], u'ortholog.pig.symbol': [u'<em>SOD1</em>']}, u'score': 36.152378, u'data': {u'ortholog': {u'fly': [{u'symbol': u'Sod', u'name': u'Superoxide dismutase', u'id': [u'P61851', u'392', u'FBpp0305736', u'EOG776SR4', u'ENOG410A6JU', u'FBpp0075958', u'FBgn0003462']}, {u'symbol': u'Ccs', u'name': u'Copper chaperone for superoxide dismutase', u'id': [u'EOG776SR4']}, {u'symbol': u'Sod

## how many diseases in each therapeutic area are associated with each target
Example of list of targets:
[u'ENSG00000119660', u'ENSG00000161671', u'ENSG00000235289', u'ENSG00000160883', u'ENSG00000105392', u'ENSG00000228966', u'ENSG00000225670', u'ENSG00000162384', u'ENSG00000146013']

In [ ]:
some_genes1=[u'ENSG00000119660', u'ENSG00000161671', u'ENSG00000235289', u'ENSG00000160883', u'ENSG00000105392', u'ENSG00000228966', u'ENSG00000225670', u'ENSG00000162384', u'ENSG00000146013']

In [28]:
r = requests.post(API + '/public/association/filter', json = {'target':random_genes,'facets':True})

In [29]:
RenderJSON(r.json())

In [30]:
therapeuticareas = []

for bucket in r.json()['facets']['therapeutic_area']['buckets']:
    therapeuticareas.append({
            'target_count' : bucket['unique_target_count']['value'], 
            'disease_count' : bucket['unique_disease_count']['value'],
            'therapeutic_area' : bucket['label'],
            'key' : bucket['key']
        })
    



Sort by target count and then disease count

In [31]:
therapeuticareas = sorted(therapeuticareas, key=lambda k: (k['target_count'],k['disease_count']), reverse=True) 

Using the python tabulate library to render a pretty table of our extracted therapeutic areas. Note: You may need to run pip install tabulate in your python environment

In [32]:
print tabulate(therapeuticareas, headers="keys", tablefmt="grid")

+------------------------------+-----------------+-------------+----------------+
| therapeutic_area             |   disease_count | key         |   target_count |
+==============================+=================+=============+================+
| neoplasm                     |              71 | efo_0000616 |              6 |
+------------------------------+-----------------+-------------+----------------+
| digestive system disease     |              33 | efo_0000405 |              6 |
+------------------------------+-----------------+-------------+----------------+
| immune system disease        |              28 | efo_0000540 |              5 |
+------------------------------+-----------------+-------------+----------------+
| eye disease                  |             109 | efo_0003966 |              4 |
+------------------------------+-----------------+-------------+----------------+
| hematological system disease |              49 | efo_0005803 |              4 |
+---------------

### TODO:
* Show the weight of genetic evidence behind the link (divide in mendelian vs gwas)
* displays list of top 3 diseases, name and Association Score.  
* show number of disease associations

* show pathway and .groupby pathway so that you bring the genes with shared biology at the top
